In [ ]:
# -*- coding: utf-8 -*-
"""
Naver Finance 뉴스 → 통합·정제·중복 제거 → news_clean_YYYYMMDD.csv
"""
import glob, re, unicodedata, pandas as pd
from collections import Counter

# ───────────────────────────────
# 1. 파일 로드
# ───────────────────────────────
def load_csvs(pattern="/Users/yujimin/KB AI CHALLENGE/project/data/news_raw/*.csv") -> pd.DataFrame:
    files = glob.glob(pattern)
    print(f"📂 Found {len(files)} CSV file(s)")
    return pd.concat((pd.read_csv(f, encoding="utf-8") for f in files), ignore_index=True)

# ───────────────────────────────
# 2. 텍스트 정규화
# ───────────────────────────────
URL_RE = re.compile(r"https?://\S+")

def normalize(text: str) -> str:
    text = unicodedata.normalize("NFKC", str(text))
    text = URL_RE.sub("", text)
    return re.sub(r"\s+", " ", text).strip()

# ───────────────────────────────
# 3. 전처리 & 중복 제거
# ───────────────────────────────
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["title", "content"]).drop_duplicates(subset=["link"])

    df["title"]   = df["title"].apply(normalize)
    df["content"] = df["content"].apply(normalize)
    df["text"]    = df["title"] + " [SEP] " + df["content"]

    # (선택) 동일 제목·날짜 중복 제거 – 본문 길이 긴 기사 우선
    df["len"] = df["content"].str.len()
    df = (
        df.sort_values("len", ascending=False)
          .drop_duplicates(subset=["title", "datetime"], keep="first")
          .drop(columns="len")
    )
    return df[["stock_name", "datetime", "text", "link"]]

# ───────────────────────────────
# 4. 저장 파일명 결정 로직
# ───────────────────────────────
def decide_filename(df: pd.DataFrame, out_dir="/Users/yujimin/KB AI CHALLENGE/project") -> str:
    # datetime 컬럼에서 ‘YYYY.MM.DD’ 부분만 추출
    dates = df["datetime"].astype(str).str.extract(r"(\d{4}\.\d{2}\.\d{2})")[0]
    # 가장 이른 날짜(earliest) 선택 — 필요 시 latest/most common 으로 교체
    target = pd.to_datetime(dates, format="%Y.%m.%d").min().strftime("%Y%m%d")
    return f"{out_dir}/news_clean_{target}.csv"

# ───────────────────────────────
# 5. 메인
# ───────────────────────────────
if __name__ == "__main__":
    raw_df   = load_csvs()
    clean_df = preprocess(raw_df)
    print(f"✅ After cleaning: {clean_df.shape}")

    out_csv = decide_filename(clean_df)
    clean_df.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"🎉 Saved → {out_csv}")

In [ ]:
from google.colab import drive
from IPython.display import clear_output
drive.mount('/content/drive')
#!pip install "transformers>=4.41.0" "torch>=2.2.0" pandas
clear_output()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
news_clean_250805.csv → KoBART 재요약(최종 요약 길이: 단어 80~110 토큰 타깃) → text 컬럼 제거
출력: news_summary_250805_len80_110.csv
"""

from __future__ import annotations
import math
from pathlib import Path

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

# ───────────────────────────────
# 1) 경로·하이퍼파라미터
# ───────────────────────────────
# Colab 예시
IN_CSV  = "/content/drive/MyDrive/Invemotion/news_clean_20250805.csv"
OUT_CSV = "/content/drive/MyDrive/Invemotion/news_summary_20250805_len80_110.csv"

# 로컬 예시
# IN_CSV  = "/Users/yujimin/KB AI CHALLENGE/project/news_clean_250805.csv"
# OUT_CSV = "/Users/yujimin/KB AI CHALLENGE/project/news_summary_250805_len80_110.csv"

DEVICE          = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE      = 8            # T4(16GB)에서 8~16 권장
MAX_INPUT_LEN   = 512          # 인코더 입력 최대 토큰
CHUNK_TOKENS    = 400          # 긴 문서 슬라이딩 윈도우 조각 크기
OVERLAP_TOKENS  = 80           # 조각 간 겹침

# ── 최종 요약(단어) 목표 범위 → BPE 환산 ─────────────────────────
# 경험치: 한국어 KoBART에서 "단어 토큰 1개 ≈ BPE 1.8개" 가량
BPE_PER_WORD        = 1.8
FINAL_WORD_RANGE    = (80, 110)    # 최종 요약 목표(단어 기준)
CHUNK_WORD_RANGE    = (50, 80)     # 조각 요약 목표(단어 기준, 메타요약 전 단계)

FINAL_MIN_NEW = int(FINAL_WORD_RANGE[0] * BPE_PER_WORD)   # ≈ 144
FINAL_MAX_NEW = int(FINAL_WORD_RANGE[1] * BPE_PER_WORD)   # ≈ 198

CHUNK_MIN_NEW = int(CHUNK_WORD_RANGE[0] * BPE_PER_WORD)   # ≈ 90
CHUNK_MAX_NEW = int(CHUNK_WORD_RANGE[1] * BPE_PER_WORD)   # ≈ 144

# ───────────────────────────────
# 2) 모델 로드
# ───────────────────────────────
print("🔄  Loading KoBART summarizer …")
tok   = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization").to(DEVICE)
if DEVICE == "cuda":
    model = model.to(dtype=torch.float16)  # VRAM 절약
model.eval()

@torch.inference_mode()
def summarize_batch(
    texts: list[str],
    min_new_tokens: int,
    max_new_tokens: int
) -> list[str]:
    """KoBART 배치 요약 (token_type_ids 제거, 길이 타깃 지정)"""
    enc = tok(
        texts,
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    enc.pop("token_type_ids", None)
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    outs = model.generate(
        **enc,
        min_new_tokens=min_new_tokens,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        length_penalty=1.2,        # 약간 더 길게 유도
        no_repeat_ngram_size=3,    # 반복 감소
        early_stopping=True
    )
    return tok.batch_decode(outs, skip_special_tokens=True)

def chunk_by_tokens(text: str,
                    chunk_tokens=CHUNK_TOKENS,
                    overlap=OVERLAP_TOKENS) -> list[str]:
    """토큰 단위 슬라이싱 → 부분 텍스트 리스트 반환"""
    ids = tok.encode(text, add_special_tokens=True)
    chunks = []
    step = max(1, chunk_tokens - overlap)
    for s in range(0, len(ids), step):
        piece = ids[s:s + chunk_tokens]
        if not piece:
            break
        chunks.append(tok.decode(piece, skip_special_tokens=True))
        if s + chunk_tokens >= len(ids):
            break
    return chunks

def summarize_long(text: str) -> str:
    """긴 문서: 조각 요약(짧게) → 메타 요약(최종 길이 타깃)"""
    parts = chunk_by_tokens(text)
    if not parts:
        return ""
    # 1차: 조각별 요약 (조각은 짧게 압축)
    part_sums = []
    for i in range(0, len(parts), BATCH_SIZE):
        part_sums += summarize_batch(parts[i:i + BATCH_SIZE],
                                     min_new_tokens=CHUNK_MIN_NEW,
                                     max_new_tokens=CHUNK_MAX_NEW)
    # 2차: 요약들의 요약(최종 길이 타깃으로 재요약)
    final = summarize_batch([" ".join(part_sums)],
                            min_new_tokens=FINAL_MIN_NEW,
                            max_new_tokens=FINAL_MAX_NEW)[0]
    return final

# ───────────────────────────────
# 3) 데이터 로드
# ───────────────────────────────
df = pd.read_csv(IN_CSV, encoding="utf-8")
assert "text" in df.columns, "입력 CSV에 'text' 컬럼이 필요합니다."
df["text"] = df["text"].fillna("")

print(f"📂 Loaded {len(df):,} rows")

# ───────────────────────────────
# 4) 512토큰 초과 여부 측정
# ───────────────────────────────
enc = tok(df["text"].tolist(), truncation=False, padding=False)
df["tok_len"]   = [len(x) for x in enc["input_ids"]]
df["truncated"] = df["tok_len"] > MAX_INPUT_LEN
print(f"⚙️  >512 tokens: {df['truncated'].mean():.1%}")

# ───────────────────────────────
# 5) 요약 실행 (최종 80~110 '단어' 토큰 타깃)
#  - 짧은 문서(≤512): 길이 타깃으로 배치 요약
#  - 긴 문서(>512):  슬라이딩 윈도우 → 메타 요약(길이 타깃)
# ───────────────────────────────
summaries = [""] * len(df)

# 5-a) 짧은 문서 일괄 처리
short_idx = df.index[~df["truncated"]].tolist()
for i in tqdm(range(0, len(short_idx), BATCH_SIZE), desc="Short docs"):
    batch_ids = short_idx[i:i + BATCH_SIZE]
    texts = df.loc[batch_ids, "text"].tolist()
    outs = summarize_batch(texts,
                           min_new_tokens=FINAL_MIN_NEW,
                           max_new_tokens=FINAL_MAX_NEW)
    for j, idx in enumerate(batch_ids):
        summaries[idx] = outs[j]

# 5-b) 긴 문서 개별 처리 (품질 우선)
long_idx = df.index[df["truncated"]].tolist()
for idx in tqdm(long_idx, desc="Long docs"):
    summaries[idx] = summarize_long(df.at[idx, "text"])

# ───────────────────────────────
# 6) 저장
# ───────────────────────────────
df.insert(2, "summary", summaries)
df = df.drop(columns=["text", "tok_len", "truncated"])

out_path = Path(OUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"🎉 Saved → {out_path}  ({len(df):,} rows, cols: {list(df.columns)})")

In [ ]:
# ⚙️ Minimal prep for KR-FinBert
import pandas as pd, unicodedata as ud, re
from transformers import AutoTokenizer

IN  = "/Users/yujimin/KB AI CHALLENGE/project/src/news_summary_20250805_len80_110.csv"
OUT = "/Users/yujimin/KB AI CHALLENGE/project/src/news_summary_20250805_len80_110_clean.csv"

df = pd.read_csv(IN)
assert set(["stock_name","datetime","summary","link"]).issubset(df.columns)

def clean_summary(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = ud.normalize("NFKC", s)          # 폭/조합 정규화
    s = s.replace("[SEP]", " ")          # 혹시 남아있다면 제거
    s = re.sub(r"\s+", " ", s).strip()   # 공백 정리
    if s and not re.search(r'[.!?\"”’)\]]$', s):  # 종결부호 보정(없을 때만)
        s += "."
    return s

df["summary"] = df["summary"].apply(clean_summary)

# (선택) 링크 기준 중복 제거 – 이미 제거되어 있으면 변화 없음
df = df.drop_duplicates(subset=["link"]).reset_index(drop=True)

# ✅ 512 토큰 초과 여부 점검(이상치 탐지 용)
tok = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
enc = tok(df["summary"].tolist(), truncation=False, padding=False)
over512 = sum(len(ids) > 512 for ids in enc["input_ids"])
print(f"Over 512 tokens: {over512} rows")

df.to_csv(OUT, index=False, encoding="utf-8-sig")
print(f"Saved → {OUT} ({len(df)} rows)")

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
KR-FinBERT-SC 감성 추론
입력:  news_summary_20250805_len80_110_clean.csv (stock_name, datetime, summary, link)
출력:  news_20250805_sent_scored.csv (neg/neu/pos + pred_label/pred_conf 추가)
"""

from __future__ import annotations
import math
from pathlib import Path

import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ───────────────────────────────────────────────────────────────
# 1) 경로/환경
# ───────────────────────────────────────────────────────────────
IN_CSV  = "/content/drive/MyDrive/Invemotion/news_summary_20250805_len80_110_clean.csv"
OUT_CSV = "/content/drive/MyDrive/Invemotion/news_20250805_sent_scored.csv"

MODEL_NAME = "snunlp/KR-FinBert-SC"   # 금융 뉴스 감성 분류 3-클래스 (neg/neu/pos)
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32                        # T4(16GB) 32~64 권장 / CPU면 8~16로 낮추세요
MAX_LEN    = 512

# ───────────────────────────────────────────────────────────────
# 2) 로드
# ───────────────────────────────────────────────────────────────
print("🔄 Loading model/tokenizer…")
tok  = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(DEVICE)
if DEVICE == "cuda":
    model = model.to(dtype=torch.float16)  # VRAM 절감
model.eval()

# id2label 안전 매핑 (모델 설정에서 직접 가져오기)
id2label = {int(i): lab for i, lab in model.config.id2label.items()}
label_list = [id2label[i].lower() for i in range(model.config.num_labels)]
# 표준화된 열 이름 매핑(모델 라벨명이 대소문자/표기 다를 수 있어 보정)
norm = {"negative":"neg", "neg":"neg",
        "neutral":"neu",  "neu":"neu",
        "positive":"pos", "pos":"pos"}
out_cols = [norm.get(l, l) for l in label_list]  # 예: ['neg','neu','pos']

# ───────────────────────────────────────────────────────────────
# 3) 데이터
# ───────────────────────────────────────────────────────────────
df = pd.read_csv(IN_CSV, encoding="utf-8")
assert set(["stock_name","datetime","summary","link"]).issubset(df.columns)
texts = df["summary"].fillna("").astype(str).tolist()
print(f"📂 Loaded {len(df):,} rows")

# ───────────────────────────────────────────────────────────────
# 4) 배치 추론
# ───────────────────────────────────────────────────────────────
def softmax_np(x):
    import numpy as np
    e = np.exp(x - x.max(axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)

probs_all = []
pred_idx_all = []

with torch.inference_mode():
    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Scoring"):
        batch = texts[i:i+BATCH_SIZE]
        enc = tok(batch, truncation=True, max_length=MAX_LEN,
                  padding=True, return_tensors="pt")
        # token_type_ids는 모델에 따라 무시되지만 안전하게 제거
        enc.pop("token_type_ids", None)
        enc = {k: v.to(DEVICE) for k, v in enc.items()}

        logits = model(**enc).logits  # [B, 3]
        if DEVICE == "cuda":
            logits = logits.float()   # softmax 위해 fp32로 임시 변환
        probs = torch.softmax(logits, dim=-1).cpu().numpy()

        pred_idx = probs.argmax(axis=1)
        probs_all.append(probs)
        pred_idx_all.append(pred_idx)

import numpy as np
probs_all = np.vstack(probs_all)              # shape [N, 3]
pred_idx_all = np.concatenate(pred_idx_all)   # shape [N]

# ───────────────────────────────────────────────────────────────
# 5) 결과 프레임 구성
# ───────────────────────────────────────────────────────────────
# 라벨 순서에 맞춰 열 이름 지정
probs_df = pd.DataFrame(probs_all, columns=out_cols)
pred_labels = [label_list[i] for i in pred_idx_all]  # 모델 원라벨(소문자)
pred_conf   = probs_all.max(axis=1)

out = pd.concat([df[["stock_name","datetime","summary","link"]],
                 probs_df], axis=1)
out["pred_label"] = [norm.get(l, l) for l in pred_labels]  # 'neg/neu/pos'로 통일
out["pred_conf"]  = pred_conf

# ───────────────────────────────────────────────────────────────
# 6) 저장
# ───────────────────────────────────────────────────────────────
out_path = Path(OUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"🎉 Saved → {out_path}  (rows={len(out):,})")
print(f"   Columns: {list(out.columns)}")
print("   Label order from model:", label_list, "-> mapped to:", out_cols)
